In [9]:
import json
from pathlib import Path
import shutil
import datetime as dt
from pprint import pprint

import pandas as pd

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


In [10]:
"""
This demo runs predictions on a pre-trained alpha model in ./tests/fixtures/alfa_exp_dir.
"""
exp_dir = PROJECT_DIRECTORY / "tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1"
with open(exp_dir / "configs.json", "r") as f:
    configs = json.load(f)

print(configs)

{'data_path': '../../data/STM Campus', 'feature_extraction': 'no', 'scenario_filter': 'no', 'scenario_wrapper': 'no', 'metric_relevancy': 'none', 'metric_wrapper': 'Interval Score', 'num_cores_wrapper': 1, 'mi_rand_frac': 1.0, 'threshold_basis': 'percentile', 'threshold_relevancy': 0.0, 'threshold_redundancy': 1.0, 'target_clean_size_weeks': 1, 'code_dir': '/projects/intelcamp/repos/WattChoice', 'results_dir': '/projects/intelcamp/repos/WattChoice/results/FTLB', 'generate_plot': True, 'generate_plot_inputdata': True, 'data_input': {'data_dir': '/projects/intelcamp/repos/WattChoice/../../data/STM Campus', 'data_config': 'FTLB Config.json', 'start_time': '2020-07-01T00:00:00-07:00', 'end_time': '2023-04-01T00:00:00-07:00', 'predictor_columns': [], 'target_var': 'FTLB CHW Meter CHW Energy Rate'}, 'data_output': {'exp_dir': '/projects/intelcamp/repos/WattChoice/results/FTLB/FTLB_FTLBCHWMeterCHWEnergyRate_r1', 'plot_comparison': True, 'plot_comparison_portion_start': 0.0, 'plot_comparison_p

In [11]:
"""
Init the model with it's configs. Be sure to switch it's use case to prediction.
"""

configs["data_output"]["exp_dir"] = exp_dir
configs["learning_algorithm"]["use_case"] = "prediction"
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)

model

Logging to: /Users/jsmith2/Code/IC/Wattile/tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1/output.out, PID: 98644


In [12]:
"""
If we want to get predictions for only one row, we have to find the time range needed to predict the nominal time.

we can use `get_input_window_for_output_time` to figure this out.
"""
nominal_time = pd.Timestamp(year=2023, month=1, day=1, tz=dt.timezone.utc)

start_time, end_time = model.get_input_window_for_output_time(nominal_time)
freq = model.configs["data_processing"]["resample"]["bin_interval"]
inclusive = model.configs["data_processing"]["resample"]["bin_label"]

predictors = model.configs["data_input"]["predictor_columns"]
target_var = model.configs["data_input"]["target_var"]

print(f"""
    To use this model to predict {target_var} for {nominal_time}, we will need to feed it data:
        - starting at {start_time}
        - ending at {end_time}
        - a rough frequency of {freq}
        - predictors named {predictors}
""")


    To use this model to predict FTLB CHW Meter CHW Energy Rate for 2023-01-01 00:00:00+00:00, we will need to feed it data:
        - starting at 2022-12-31 18:00:00+00:00
        - ending at 2023-01-01 00:15:00+00:00
        - a rough frequency of 15min
        - predictors named []



In [13]:
"""
Great, now lets make some dummy data that matches those specs.
"""

data = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=freq, inclusive=inclusive))
for predictor in predictors:
    data[predictor] = data.index.hour * 100 + data.index.minute

# even though we are predicting, we must fill the target column with dummy data. It's not used though.
data[target_var] = data.index.hour * 100 + data.index.minute


data

,FTLB CHW Meter CHW Energy Rate
2022-12-31 18:00:00+00:00,1800
2022-12-31 18:15:00+00:00,1815
2022-12-31 18:30:00+00:00,1830
2022-12-31 18:45:00+00:00,1845
2022-12-31 19:00:00+00:00,1900
2022-12-31 19:15:00+00:00,1915
2022-12-31 19:30:00+00:00,1930
2022-12-31 19:45:00+00:00,1945
2022-12-31 20:00:00+00:00,2000
2022-12-31 20:15:00+00:00,2015


In [14]:
"""
Let's prep those for the model.
"""
train_df, val_df = prep_for_rnn(configs, data)
val_df

,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,HOD_binary_reg_1_min_lag24,HOD_binary_reg_2_min_lag24,HOD_binary_reg_3_min_lag24,HOD_binary_reg_4_min_lag24,HOD_binary_reg_5_min_lag24,HOD_binary_reg_6_min_lag24,HOD_binary_reg_7_min_lag24,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,FTLB CHW Meter CHW Energy Rate
2023-01-01 00:00:00+00:00,-1.0,-1.836970e-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.017213,0.999852,0


In [15]:
"""
One line in, one prediction (with many quantiles) out.
"""
results = model.predict(val_df)

results

RuntimeError: Error(s) in loading state_dict for LSTM_Model:
	size mismatch for lstm.weight_ih_l0: copying a param with shape torch.Size([100, 237]) from checkpoint, the shape in current model is torch.Size([100, 198]).